[WordAndDoc2vec](https://github.com/darecophoenixx/wordroid.sblo.jp/wiki/WordAndDoc2vec)

In [ ]:
!pip install japanize-matplotlib

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import random

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'retina'

In [ ]:
import japanize_matplotlib
plt.rcParams['font.family']

In [ ]:
img_cnt = 0

In [ ]:
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.png'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name)

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

In [ ]:
df_part = pd.read_csv('../input/ks2020-encode-data-001/encoded_01.csv', index_col=0)
df_part.head()

In [ ]:
df_out_wgt_user = pd.read_csv('../input/ks2020-lowcol-fet12-001/out_user.csv', index_col=0)
X_df_user = df_out_wgt_user
wgt_user = df_out_wgt_user.values
df_out_wgt_user.head()

In [ ]:
df_out_wgt_col = pd.read_csv('../input/ks2020-lowcol-fet12-001/out_col.csv', index_col=0)
X_df_col = df_out_wgt_col
wgt_prod = df_out_wgt_col.values
df_out_wgt_col.head()

In [ ]:
X_df_user_som_out = pd.read_csv('../input/ks2020-lowcol-fet12-001-somcls-sksom-keras/som_out_user.csv', index_col=0)
X_df_user_som_out.head()

In [ ]:
X_df_col_som_out = pd.read_csv('../input/ks2020-lowcol-fet12-001-somcls-sksom-keras/som_out_col.csv', index_col=0)
X_df_col_som_out.head()

In [ ]:
X_df_lm = pd.read_csv('../input/ks2020-lowcol-fet12-001-somcls-sksom-keras/som_out_LM.csv', index_col=0)
X_df_lm.head()

In [ ]:
X_df = df_out_wgt_user

In [ ]:
sns.pairplot(X_df.iloc[:,:5], markers=['.'], height=2.2)
save_img()

In [ ]:
ix, jx = '0', '1'

plt.figure(figsize=(12,12))
ax = sns.kdeplot(data=X_df, x=ix, y=jx,
                 cmap="Blues", shade=True, thresh=0.05)
ax.scatter(X_df_lm.values[:,int(ix)], X_df_lm.values[:,int(jx)], marker='X', s=100, c='orange')

for ii, col in enumerate(X_df_lm.columns):
    text = 'CLASS_' + str(col)
    xy = (X_df_lm.values[ii,int(ix)], X_df_lm.values[ii,int(jx)])
    ax.annotate(text, xy=xy, size=16)
save_img()

In [ ]:
'''列（ブランドや属性）がどのクラスに属したか'''
enc = preprocessing.OneHotEncoder()
enc.fit(X_df_col_som_out['cls'].values.reshape(-1,1))
tmp = pd.DataFrame(enc.transform(X_df_col_som_out['cls'].values.reshape(-1,1)).toarray().astype(int), index=X_df_col_som_out.index.values)
tmp.columns = ['cls'+str(ee) for ee in range(12)]
tmp

In [ ]:
'''各クラスのユーザー数'''
cnt = pd.DataFrame({'cls': X_df_user_som_out['cls']}).groupby('cls').size()
cnt

In [ ]:
'''[クラス]×[列の各変数] クロス集計'''
ll = []
for ee in X_df_col_som_out.index.values:
    tmp = pd.crosstab(X_df_user_som_out['cls'], df_part[ee], margins=True, normalize='index')
    #print(tmp)
    ll.append(tmp.values[:,1])

res = pd.DataFrame(np.c_[ll].T, index=tmp.index, columns=df_part.columns)
res.iloc[:,:10]

In [ ]:
res.iloc[:,10:20]

In [ ]:
res.iloc[:,20:30]

In [ ]:
res.iloc[:,30:40]

In [ ]:
cnt1 = cnt.values
cnt1 = np.r_[cnt1, cnt.values.sum()]
cnt1 = pd.DataFrame(cnt1)
cnt1.index = res.index.values
cnt1
res1 = pd.concat([res, pd.DataFrame(cnt1)], axis=1)
res1.columns.values[-1] = 'cnt'
res1

In [ ]:
res1.to_csv('som_cls_crosstab.csv')